In [1]:
import pandas as pd
import re
from tqdm import tqdm

import matplotlib.pyplot as plt
import numpy as np
import locationtagger



import nltk
import spacy
 
# essential entity models downloads
nltk.downloader.download('maxent_ne_chunker')
nltk.downloader.download('words')
nltk.downloader.download('treebank')
nltk.downloader.download('maxent_treebank_pos_tagger')
nltk.downloader.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /Users/gopalsingh/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     /Users/gopalsingh/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package treebank to
[nltk_data]     /Users/gopalsingh/nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package maxent_treebank_pos_tagger to
[nltk_data]     /Users/gopalsingh/nltk_data...
[nltk_data]   Package maxent_treebank_pos_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/gopalsingh/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/gopalsingh/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
df = pd.read_csv("google_review_29_08_v3.csv")
df.head()

,SVAAS_ID,user_id,firstName,middleName,lastName,email,phone,description,pan,mci,...,numberFollowUp,followUpDuration,insuranceProviderId,isActive,ABDM_Healthcare_Professional_ID,SVAAS_DOCTOR_ID,svaasClinicId,tbl_user_phone_number,name,location
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df = df[['SVAAS_ID','firstName', 'middleName', 'lastName', 'name', 'baseCity']]
df.head()

,SVAAS_ID,firstName,middleName,lastName,name,baseCity
0,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN


### I think, it will be safe to drop the middleName column, and we can proceed with the firstName only here.

In [4]:
df.drop('middleName', axis=1, inplace=True)

## Let's seggregate the data based on missing vs non-missing row values

In [5]:
df_not_missing = df.dropna()

In [6]:
df_not_missing.head()

,SVAAS_ID,firstName,lastName,name,baseCity
17,3.0,Divya,Kavuri,Apollo Clinic,Hyderabad
18,4.0,Shivam,P,Shivam hospitals,Hyderabad
19,5.0,Ranjan,Dey,Ranjan skin clinics,Hyderabad
20,6.0,Soumya,Sarkar,Soumya Multi speciality clinic,Hyderabad
21,7.0,Rishanku,Goyal,RKG Eye Center,Hyderabad


In [7]:
df_not_missing.isna().sum()

SVAAS_ID     0
firstName    0
lastName     0
name         0
baseCity     0
dtype: int64

In [8]:
df_not_missing.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1715 entries, 17 to 1735
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   SVAAS_ID   1715 non-null   float64
 1   firstName  1715 non-null   object 
 2   lastName   1715 non-null   object 
 3   name       1715 non-null   object 
 4   baseCity   1715 non-null   object 
dtypes: float64(1), object(4)
memory usage: 80.4+ KB


In [9]:
print(len(df_not_missing))

1715


In [10]:
len(df_not_missing)/len(df)

0.9879032258064516

In [11]:
df_not_missing['query'] = df_not_missing['firstName'].apply(lambda x:x.lower()) +"+" + df_not_missing['lastName'].apply(lambda x:x.lower()) \
                          +"+"+ df_not_missing['name'].apply(lambda x:x.lower()) + "+" + df_not_missing["baseCity"].apply(lambda x:x.lower())

/var/folders/z1/0tn2v83n6pzgl2m8lzz1phbc0000gn/T/ipykernel_38066/4047833984.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_not_missing['query'] = df_not_missing['firstName'].apply(lambda x:x.lower()) +"+" + df_not_missing['lastName'].apply(lambda x:x.lower()) \


In [12]:
df_not_missing['query']

17                   divya+kavuri+apollo clinic+hyderabad
18                    shivam+p+shivam hospitals+hyderabad
19               ranjan+dey+ranjan skin clinics+hyderabad
20      soumya+sarkar+soumya multi speciality clinic+h...
21                rishanku+goyal+rkg eye center+hyderabad
                              ...                        
1731    kushang+mod+shivam children hospital and neona...
1732           dhaval+akhani+childhood hospital+ahmedabad
1733    keval patel+patel+jk dental hospital orthodont...
1734    jinal patel+patel+jk dental hospital orthodont...
1735                   lavanya+yasam+sai clinic+hyderabad
Name: query, Length: 1715, dtype: object

## For missing data

In [13]:
import preprocess_kgptalkie as ps
import re

def get_clean(x):
    x = str(x).lower().replace('\\', '').replace('_', ' ')
    x = ps.cont_exp(x)
    x = ps.remove_emails(x)
    x = ps.remove_urls(x)
    x = ps.remove_html_tags(x)
    x = ps.remove_rt(x)
    x = ps.remove_accented_chars(x)
    x = ps.remove_special_chars(x)
    x = re.sub("(.)\\1{2,}", "\\1", x)
    return x




In [14]:
df_not_missing['query'] = df_not_missing['query'].apply(lambda x:ps.remove_accented_chars(x))
df_not_missing['query'].head()

/var/folders/z1/0tn2v83n6pzgl2m8lzz1phbc0000gn/T/ipykernel_38066/153722280.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_not_missing['query'] = df_not_missing['query'].apply(lambda x:ps.remove_accented_chars(x))


17                 divya+kavuri+apollo clinic+hyderabad
18                  shivam+p+shivam hospitals+hyderabad
19             ranjan+dey+ranjan skin clinics+hyderabad
20    soumya+sarkar+soumya multi speciality clinic+h...
21              rishanku+goyal+rkg eye center+hyderabad
Name: query, dtype: object

In [15]:
len(df_not_missing['query'].values)

1715

In [16]:
from collections import defaultdict

final_df = defaultdict(list)

In [17]:
df_not_missing

,SVAAS_ID,firstName,lastName,name,baseCity,query
17,3.0,Divya,Kavuri,Apollo Clinic,Hyderabad,divya+kavuri+apollo clinic+hyderabad
18,4.0,Shivam,P,Shivam hospitals,Hyderabad,shivam+p+shivam hospitals+hyderabad
19,5.0,Ranjan,Dey,Ranjan skin clinics,Hyderabad,ranjan+dey+ranjan skin clinics+hyderabad
20,6.0,Soumya,Sarkar,Soumya Multi speciality clinic,Hyderabad,soumya+sarkar+soumya multi speciality clinic+h...
21,7.0,Rishanku,Goyal,RKG Eye Center,Hyderabad,rishanku+goyal+rkg eye center+hyderabad
...,...,...,...,...,...,...
1731,2425.0,Kushang,Mod,Shivam Children Hospital And Neonatal Care Centre,Ahmedabad,kushang+mod+shivam children hospital and neona...
1732,2426.0,Dhaval,Akhani,Childhood Hospital,Ahmedabad,dhaval+akhani+childhood hospital+ahmedabad
1733,2427.0,Keval Patel,Patel,Jk Dental Hospital Orthodontic & Implant Center,Ahmedabad,keval patel+patel+jk dental hospital orthodont...
1734,2428.0,Jinal Patel,Patel,Jk Dental Hospital Orthodontic & Implant Center,Ahmedabad,jinal patel+patel+jk dental hospital orthodont...


In [18]:
# df_ch = df_not_missing[df_not_missing['baseCity'] == 'Chennai']
# df_ch.isna().sum()

In [19]:
def isDigit(char):
    pattern = "^[-+]?[0-9]*\.?[0-9]+(e[-+]?[0-9]+)?$"
    reg = re.compile(pattern)
    if reg.match(char):
        return True
    return False

In [20]:
# from bs4 import BeautifulSoup
# import urllib.request
# import numpy as np

# def extractReviews(df_not_missing, limit=100):
#     queries = df_not_missing['query'].values[:limit]

#     for i in tqdm(range(len(queries))):
#         query = queries[i]
#         query = "+".join(query.split(" "))
#         url = 'https://google.com/search?q='+query
#         print(url)

#         request = urllib.request.Request(url)
#         request.add_header('User-Agent', 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36')
#         raw_response = urllib.request.urlopen(request).read()

#         # Read the repsonse as a utf-8 string
#         html = raw_response.decode("utf-8")

#         soup = BeautifulSoup(html, 'html.parser')

#         try:
#             rating = soup.find("span", class_="Aq14fc")
#             review = soup.find("span", class_="hqzQac")
#             address = soup.find("span", class_="LrzXr").get_text()
#             phone = soup.find("span", class_="LrzXr zdqRlf kno-fv").get_text()
#             speciality = soup.find("span", class_="YhemCb").get_text()
#             working_hours = soup.find("table", class_="WgFkxc").get_text()
            
#             print("address: ", address)
#             print("phone: ", phone)
#             print("spec: ", speciality)

#             rating = rating.get_text()
#             greviews = review.get_text()
#             print(greviews)
#             greviews = [ps.remove_special_chars(i) for i in greviews.split(" ")]
#             greviews = [i for i in greviews if isDigit(i)][0]
            
            
#             final_df['firstName'].append(df_not_missing.iloc[i]['firstName'])
#             final_df['lastName'].append(df_not_missing.iloc[i]['lastName'])
#             final_df['clinicName'].append(df_not_missing.iloc[i]['name'])
#             final_df['baseCity'].append(df_not_missing.iloc[i]['baseCity'])
#             final_df['SVAAS_ID'].append(df_not_missing.iloc[i]['SVAAS_ID'])

#             if greviews:
#                 final_df['GoogleNRs'].append(float(greviews))
#                 print("GNumber of google reviews: ", greviews)
#             else:
#                 final_df['GoogleNRs'].append("NA")
#                 print("Google reviews are not available.")
            
#             if working_hours:
#                 final_df['workingHours'].append(working_hours)
#                 print("working hours: ", working_hours)
#             else:
#                 print("Working hours are not available.")
#                 final_df['workingHours'].append("NA")
            
            

#             if rating:
#                 final_df['rating'].append(rating)
#                 print("Rating is :", rating)
#             else:
#                 final_df['rating'].append("NA")
#                 print("Rating is not available.")

#             if address:
#                 final_df['address'].append(address)
#                 print("Address: ", address)
#             else:
#                 final_df['address'].append("NA")
#                 print("Address is not available.")

#             if phone:
#                 final_df['phoneNo'].append(phone)
#                 print("Phone number: ", phone)
#             else:
#                 final_df['phoneNo'].append("NA")
#                 print("Phone number is not availble.")


#             if speciality:
#                 final_df['speciality'].append(speciality)
#                 print("Speciality: ", speciality)
#             else:
#                 final_df['speciality'].append("NA")
#                 print("Speciality not available")


#             text = ps.remove_accented_chars(address)
#             text = ps.remove_special_chars(text)
#             text = ps.remove_html_tags(text)

#             text = text.split(" ")
#             print(text)
#             print(text[-4:][:-1])
#             city = " ".join(text[-4:][:-1])


            
#             if city:
#                 print("Extracted city: ", city)
#                 final_df['extractedCity'].append(city)
#             else:
#                 print("Couldn't extract the city.")
#                 final_df["extractedCity"].append("NA")
            
#         except Exception as e:
#             print("Error was: ", e)
                
                    
#     return final_df

#     #             print(div.get_text())


In [21]:
# final_df = extractReviews(df_not_missing, limit=5)

In [22]:
# pd.DataFrame(final_df)

### Comment Extraction

In [32]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
import time



options = Options()
options.headless = True
# As there are possibilities of different chrome
# browser and we are not sure under which it get
# executed let us use the below syntax
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

driver.get("https://www.google.com/search?q=rishanku+goyal+rkg+eye+center+hyderabad&safe=strict")
driver.find_element(By.LINK_TEXT, "180 Google reviews").click()

time.sleep(10)
exctracted_comments = []

for c in driver.find_elements(By.CLASS_NAME, "review-full-text"):
    time.sleep(5)
    print(c.get_attribute("innerText"))
    print("*"*100)
    exctracted_comments.append(c.get_attribute("innerText"))

Goyal Eye Institute is the best place for any eye problem. I’m coming here fromlong time. My Retina detachment surgery and cataract surgery were done here. I’m happy with my Surgery. Doctors are very polite and very humble. I feel very comfortable here. If you visit here, it means you are in safe hands.Thanks, Goyal Eye Institute for your marvelous services.
****************************************************************************************************
I recently had an amazing experience at Goyal Eye where my mother went through a cataract surgery. The doctor was kind, answered all our questions and patiently "allowed" us to be ready for the surgery. This was rare because usually doctors want the "package" completed promptly so they can work on the next one. Not this one... Highly recommend Dr. Anshul Goyal at Goyal Eye institute. The staff, well, they can do better; but medical expertise is par excellence! I will be going back soon for my mother's cataract surgery in the second 

In [33]:
len(exctracted_comments)

6